In [250]:
import numpy as np

In [251]:
# def sigmoid(x):
#     return 1 / (1 + np.exp(-x))

def sigmoid(x):
    if isinstance(x, list):
        sigmoids = []
        for value in x:
            sigmoids.append(sigmoid(value))
        return sigmoids
    else:
        return 1 / (1 + np.exp(-x))


def sigmoid_derivative(x):
    if isinstance(x, list):
        sigmoids_derivative = []
        for value in x:
            sigmoids_derivative.append(sigmoid_derivative(value))
        return sigmoids_derivative
    else:
        s = sigmoid(x)
        return s * (1 - s)

In [252]:
# example in the lecture 4

X = np.array([0, 0, 1])

Weights = [
    np.array([
        [0.21, 0.15, -0.3],
        [0.10, 0.25, -0.4]
    ]),
    np.array([
        [-0.2, 0.3, -0.4]
    ])
]


In [ ]:
import numpy as np

# Input vector
X = np.array([0, 0, 1])  # Include bias as the last input
y = np.array([0])

# Weights for the layers
Weights = [
    np.array([
        [0.21, 0.15, -0.3],
        [0.10, -0.4, 0.25]
    ]),
    np.array([
        [-0.2, 0.3, -0.4]
    ])
]
activations = []
sigmasList = []
# Sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Feedforward function
def feedForward(X):
    global activations, nets
    activations = [X] 
    nets = []
    for i in range(len(Weights)):
        # Compute the input for the current layer
        layer_input = np.dot(activations[i], Weights[i].T)
        nets.append(layer_input)
        # print("layer_input", layer_input)
        
        # Apply activation function (sigmoid for hidden layers, no activation for output layer)
        layer_input = sigmoid(layer_input)
        if i != len(Weights) - 1:  # If not the output layer
            # Add bias to the next layer input
            layer_input = np.append(layer_input, 1)
        
        # Append the result to the activations list
        activations.append(layer_input)
        # print("activations", activations)
        # print("nets = ", nets)
        # print("nest shape : ", np.array(nets).shape)
    
    return activations[-1]  # Return the output of the final layer


# print(feedForward(X))
# # print("nets = ", nets)
# print("activation" ,activations)
def backPropagation(X, y):
    global sigmasList

    # Feedforward
    output_activation = feedForward(X)

    sigmasList = []
    sigmaOutput = (y - output_activation) * output_activation * (1 - output_activation)
    # print("sigmaOutput", sigmaOutput)

    sigmas = sigmaOutput
    for i in range(len(Weights)-1, 0, -1):
        sigmasList.append(sigmas * Weights[i] * sigmoid_derivative(nets[i]))

    sigmasList.reverse()
    sigmasList.append(sigmaOutput)


# backPropagation(X, 0, learning_rate=0.1)
# print("Sigma" , sigmasList)
# under creation method
def ModifyWeights(learningRate = 0.1, epoch=0):
    if epoch > 1000:
        learningRate = 0.01
    for i, sigmas in enumerate(sigmasList):
        Weights[i] = Weights[i] + learningRate * sigmas * activations[i]

# print("Weights before" , Weights)      
# ModifyWeights(0.1)
# print("Weights after" , Weights)

# Input vector
X = np.array([[0, 0, 1], [0,1,1], [1,0,1], [1,1,1]])  # Include bias as the last input
y = np.array([0, 0, 0, 1])

def fit(X, y, epochs = 1000):
    print("Weights before" , Weights) 
    for epoch in range(epochs):
        total_loss = 0
        # Loop over each training sample
        for i in range(X.shape[0]):
            X_sample = X[i]
            y_sample = y[i]

            # Feedforward and backpropagation for each sample
            feedForward(X_sample)
            backPropagation(X_sample, y_sample)
            ModifyWeights(0.1)  # Update weights after each sample

            # Calculate loss (Mean Squared Error) for monitoring
            output = activations[-1]
            total_loss += np.mean((y_sample - output) ** 2)
        
        # Print the loss every 100 epochs
        if epoch % 70 == 0:
            print(f'Epoch {epoch}, Loss: {total_loss / X.shape[0]}')

    print("Weights after training:", Weights)

    #     feedForward(X)
    #     backPropagation(X, y)
    #     ModifyWeights(0.01)
    # print("Weights after" , Weights)

fit(X,y, 2000)

# fit(np.array([0, 0, 1]), np.array([0]), 400)
# fit(np.array([1, 0, 1]), np.array([0]), 600)
# fit(np.array([0, 1, 1]), np.array([0]), 300)
# fit(np.array([1, 1, 1]), np.array([1]), 500)

# fit(np.array([0, 0, 0]), np.array([0]), 400)
# fit(np.array([1, 0, 0]), np.array([0]), 600)
# fit(np.array([0, 1, 0]), np.array([0]), 300)
# fit(np.array([1, 1, 0]), np.array([1]), 500)

def predict(X, bias=True):
    # Perform a forward pass
    biasEnable = 1 if bias else 0
    X = np.append(X, biasEnable)
    output = feedForward(X)


    # for i in range(len(Weights)):
    #     print("X.shape : ", X.shape)
    #     print(f"Weights[{i}].shape : ", Weights[i].shape)
    #     biasEnable = 1 if bias else 0
    #     X = np.append(X, biasEnable)
    #     X = np.dot([X], Weights[i].T)
    #     activation = sigmoid(X)
    #     X = activation
    #     print(f"loop {i} passed!")

    # return activation

    # Predicted class: index of the highest output neuron
    predicted_class = np.argmax(output)
    return predicted_class

print(predict(np.array([0, 0])))



from sklearn.metrics import confusion_matrix

# Assuming X_train, y_train, X_test, y_test are available

def calculate_accuracy_and_confusion_matrix(X_train, y_train, X_test, y_test, bias=True):
    # Initialize counters for accuracy and confusion matrix
    correct_train = 0
    correct_test = 0
    total_train = len(X_train)
    total_test = len(X_test)
    
    # Initialize confusion matrices
    cm_train = np.zeros((3, 3), dtype=int)  # For 3 classes
    cm_test = np.zeros((3, 3), dtype=int)

    # Predictions on the training data
    for i in range(total_train):
        predicted = predict(X_train[i], bias)
        actual = y_train[i]
        if predicted == actual:
            correct_train += 1
        cm_train[actual][predicted] += 1
    
    # Predictions on the test data
    for i in range(total_test):
        predicted = predict(X_test[i], bias)
        actual = y_test[i]
        if predicted == actual:
            correct_test += 1
        cm_test[actual][predicted] += 1
    
    # Calculate accuracy
    train_accuracy = correct_train / total_train * 100
    test_accuracy = correct_test / total_test * 100

    # Print Results
    print(f"Training Accuracy: {train_accuracy:.2f}%")
    print(f"Testing Accuracy: {test_accuracy:.2f}%")
    print("Training Confusion Matrix:")
    print(cm_train)
    print("Testing Confusion Matrix:")
    print(cm_test)

# Example usage with your data
# Assuming X_train, y_train, X_test, y_test are NumPy arrays with training and test data
calculate_accuracy_and_confusion_matrix(X_train, y_train, X_test, y_test, bias=True)

    

Weights before [array([[ 0.21,  0.15, -0.3 ],
       [ 0.1 , -0.4 ,  0.25]]), array([[-0.2,  0.3, -0.4]])]
Epoch 0, Loss: 0.21867816773421128
Epoch 70, Loss: 0.190577736203999
Epoch 140, Loss: 0.1892680050245271
Epoch 210, Loss: 0.1885409218985165
Epoch 280, Loss: 0.1878078420290692
Epoch 350, Loss: 0.18699998672063067
Epoch 420, Loss: 0.18607037548151825
Epoch 490, Loss: 0.1849620138086021
Epoch 560, Loss: 0.18359999236876934
Epoch 630, Loss: 0.18188571771689083
Epoch 700, Loss: 0.17969304000314945
Epoch 770, Loss: 0.17687059535136374
Epoch 840, Loss: 0.17326029820598718
Epoch 910, Loss: 0.16874898492714172
Epoch 980, Loss: 0.1633691043054037
Epoch 1050, Loss: 0.15742783649846692
Epoch 1120, Loss: 0.15155799086332417
Epoch 1190, Loss: 0.14655512112695962
Epoch 1260, Loss: 0.14305706481970104
Epoch 1330, Loss: 0.1413290038891512
Epoch 1400, Loss: 0.141278303802803
Epoch 1470, Loss: 0.14258478660569837
Epoch 1540, Loss: 0.14482793861138965
Epoch 1610, Loss: 0.14758355717281163
Epoch 168